In [ ]:
# code modeled after https://github.com/madelinehayes/seabirdNET
# note: code is almost identical to Tile_Harbor_Seals

In [ ]:
from PIL import Image
import os
from os import walk
import argparse
import numpy as np
import json
import csv
import rasterio
from rasterio.plot import reshape_as_image
from rasterio.windows import Window
import matplotlib
import folium
from pyproj import Proj, transform

%matplotlib inline

Image.MAX_IMAGE_PIXELS = 100000000000

In [ ]:
# access all elephant seal imagery
seal = 'Elephant_Seals'
sealPath = '../Images/' + seal 

overlap = 80
only_beach = False    
    
files = next(walk(sealPath), (None, None, []))[2]  # empty if no files exist

In [ ]:
# make sure images are all the same size
def checkImageSize(seal, image):
    infile = '../Images/' + seal +'/' + image
    dataset = rasterio.open(infile)
    img_height, img_width = dataset.shape
    return img_height, img_width
    
    
sizes = set()
for image in files:
    size = checkImageSize(seal, image)
    sizes.add(size)
    
print(len(sizes))
assert(len(sizes) == 1)

In [ ]:
# look at top left corner coordinates of all tiles
if False:
    print(sizes)
    for size in sizes:
        img_height, img_width = size
        tile_height = tile_width = 500
        stride = tile_height - overlap
        for r in range(0, img_height-tile_height+1, stride):
            for c in range(0, img_width-tile_width+1, stride):
                print(r, c) #row, column
            print()

In [ ]:
def processImage(seal, image, overlap):
    '''
    args:
    seal is a string, either "Harbor_Seals" or "Elephant_Seals"
    image is the satellite image file
    overlap is an int with the size of overlap between images
    '''
    infile = '../Images/' + seal +'/' + image
    img_dir = '../Images/' + seal 
    if only_beach:
        img_dir += '_Beach_Only'
    else:
        img_dir += '_Full_Data'
    prj_name = img_dir.split("/")[-1]
    
    # open file (read), get dims
    dataset = rasterio.open(infile)
    img_height, img_width = dataset.shape
    
    # define tile size and number of pixels to move in each direction
    tile_height = tile_width = 500
    stride = tile_height - overlap
    start_num = 0
    img_dict = {}
    count = 0
    
    # create the dir if it doesn't already exist
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
        
    num_tiles = (img_width * img_height) / ((tile_height - overlap) * (tile_width - overlap))
    #print("image size", dataset.shape)
    #print("number of tiles:", num_tiles)
    
    for r in range(0, img_height-tile_height+1, stride):
        for c in range(0, img_width-tile_width+1, stride):
            # read tile
            tile = dataset.read((1,2,3),window=Window(c, r, tile_width, tile_height))
            
            reshaped_tile = reshape_as_image(tile)
            
            top_pixel = [c,r]
            imag = image.split(".")[0]
            tile_name = imag + "---" + str(count) + ".png"
            img_dict[tile_name] = top_pixel
            
            im = Image.fromarray(reshaped_tile)
            # save 
            path = os.path.join(img_dir, tile_name)
            im.save(path)
            
            count += 1
            

In [ ]:
countI = 1
for image in files:
    processImage(seal, image, overlap)
    print('finished image', countI)
    countI += 1

In [ ]:
print("done")

In [ ]:
# get images on right and bottom
def get_right_bottom_image(seal, image):
    infile = '../Images/' + seal +'/' + image
    img_dir = '../Images/' + seal 
    if only_beach:
        img_dir += '_Beach_Only'
    else:
        img_dir += '_Full_Data'
    prj_name = img_dir.split("/")[-1]
    
    # open file (read), get dims
    dataset = rasterio.open(infile)
    img_height, img_width = dataset.shape
    
    # define tile size and number of pixels to move in each direction
    tile_height = tile_width = 500
    # stride = tile_height - overlap
    start_num = 0
    img_dict = {}
    count = 0
    
    far_x = img_width - 500
    bottom_y = img_height - 500
    
    # create the dir if it doesn't already exist
    if not os.path.exists(img_dir):
        os.makedirs(img_dir)
        
    num_tiles = (img_width * img_height) / ((tile_height - overlap) * (tile_width - overlap))
    #print("image size", dataset.shape)
    #print("number of tiles:", num_tiles)
    count = int(num_tiles)
    r = 0
    c = far_x
    stride = tile_height - overlap
    # get the right most column... will have some extra overlap with previously tiled tiles
    while r + tile_height < img_height:
        tile = dataset.read((1,2,3),window=Window(c, r, tile_width, tile_height))
        
        reshaped_tile = reshape_as_image(tile)
            
        top_pixel = [c,r]
        imag = image.split(".")[0]
        tile_name = imag + "---" + str(count) + ".png"
        img_dict[tile_name] = top_pixel

        im = Image.fromarray(reshaped_tile)
        # save 
        path = os.path.join(img_dir, tile_name)
        im.save(path)

        count += 1    
        r += stride
    
    r = bottom_y
    c = 0
    # get bottom most row... again will have some extra overlap
    while c + tile_width < img_width:
        tile = dataset.read((1,2,3),window=Window(c, r, tile_width, tile_height))
        
        reshaped_tile = reshape_as_image(tile)
            
        top_pixel = [c,r]
        imag = image.split(".")[0]
        tile_name = imag + "---" + str(count) + ".png"
        img_dict[tile_name] = top_pixel

        im = Image.fromarray(reshaped_tile)
        #save 
        path = os.path.join(img_dir, tile_name)
        im.save(path)

        count += 1    
        c += stride

In [ ]:
countI = 1
for image in files:
    get_right_bottom_image(seal, image)
    print('finished image', countI)
    countI += 1

In [ ]:
print("done")